In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from config import g_api_map 

In [2]:
weather_data = pd.read_csv("output_data/cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Constitución,18,CL,1649432370,67,-35.3333,-72.4167,65.57,8.90
1,Qaanaaq,11,GL,1649432370,73,77.4840,-69.3632,-1.41,6.49
2,Talcahuano,75,CL,1649432370,82,-36.7167,-73.1167,60.48,6.91
3,Okhotsk,100,RU,1649432370,89,59.3833,143.3000,19.29,6.17
4,Kalmar,40,SE,1649432371,84,56.6616,16.3616,43.14,16.11
...,...,...,...,...,...,...,...,...,...
565,Bereket,61,TM,1649432444,39,39.2446,55.5154,70.74,4.21
566,Moroni,40,KM,1649432445,81,-11.7022,43.2551,81.30,8.01
567,Central Point,75,US,1649432445,67,42.3760,-122.9164,55.42,3.44
568,Upata,78,VE,1649432445,36,8.0086,-62.3989,91.74,7.81


In [3]:
# Configure gmaps
gmaps.configure(api_key = g_api_map )

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
max_intensity

100

In [6]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
49,Tura,0,IN,1649432377,71,25.5198,90.2201,74.37,6.24
71,Laguna,0,US,1649432380,61,38.4210,-121.4238,76.01,1.01
155,Santa Vitória do Palmar,0,BR,1649432391,54,-33.5189,-53.3681,76.48,9.86
207,Mendoza,0,AR,1649432398,73,-32.8908,-68.8272,72.93,6.91
311,Nuevo Laredo,0,MX,1649432412,37,27.4763,-99.5164,70.07,6.91
354,San Patricio,0,US,1649432417,26,28.0170,-97.5169,74.93,6.02
397,Faya,0,SA,1649432422,28,18.3851,42.4509,73.51,3.24
551,Tiznit Province,0,MA,1649432442,34,29.5833,-9.5000,72.84,9.95


In [7]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
49,Tura,IN,25.5198,90.2201,
71,Laguna,US,38.4210,-121.4238,
155,Santa Vitória do Palmar,BR,-33.5189,-53.3681,
207,Mendoza,AR,-32.8908,-68.8272,
311,Nuevo Laredo,MX,27.4763,-99.5164,
354,San Patricio,US,28.0170,-97.5169,
397,Faya,SA,18.3851,42.4509,
551,Tiznit Province,MA,29.5833,-9.5000,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_api_map}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 49: Tura.
Closest hotel in Tura is Hotel Polo Orchid.
------------
Retrieving Results for Index 71: Laguna.
Closest hotel in Laguna is Hampton Inn & Suites Sacramento-Elk Grove Laguna I-5.
------------
Retrieving Results for Index 155: Santa Vitória do Palmar.
Closest hotel in Santa Vitória do Palmar is Mirante Hotel.
------------
Retrieving Results for Index 207: Mendoza.
Closest hotel in Mendoza is Huentala Hotel.
------------
Retrieving Results for Index 311: Nuevo Laredo.
Closest hotel in Nuevo Laredo is Holiday Inn Express Nuevo Laredo, an IHG Hotel.
------------
Retrieving Results for Index 354: San Patricio.
Closest hotel in San Patricio is Americas Best Value Inn Sinton.
------------
Retrieving Results for Index 397: Faya.
Missing field/result... skipping.
------------
Retrieving Results for Index 551: Tiznit Province.
Missing field/result... skipping.
------------
-------End of Search-------


In [10]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
49,Tura,IN,25.5198,90.2201,Hotel Polo Orchid
71,Laguna,US,38.4210,-121.4238,Hampton Inn & Suites Sacramento-Elk Grove Lagu...
155,Santa Vitória do Palmar,BR,-33.5189,-53.3681,Mirante Hotel
207,Mendoza,AR,-32.8908,-68.8272,Huentala Hotel
311,Nuevo Laredo,MX,27.4763,-99.5164,"Holiday Inn Express Nuevo Laredo, an IHG Hotel"
354,San Patricio,US,28.0170,-97.5169,Americas Best Value Inn Sinton
397,Faya,SA,18.3851,42.4509,
551,Tiznit Province,MA,29.5833,-9.5000,


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))